# Attention: This code can only be successfully run under Python 2.7

In [10]:
from fmm import FastMapMatch,Network,NetworkGraph,UBODTGenAlgorithm,UBODT,FastMapMatchConfig

### Read network and graph

In [11]:
# network = Network("../data/porto/edges.shp","fid", "u", "v")
network = Network("../data/didi/edges.shp","fid", "u", "v")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))
graph = NetworkGraph(network)

Nodes 63329 edges 151935


### Precompute UBODT

In [4]:
# Can be skipped if you already generated an ubodt file
ubodt_gen = UBODTGenAlgorithm(network,graph)
# The delta is defined as 3 km approximately. 0.03 degrees. 
# status = ubodt_gen.generate_ubodt("../data/porto/ubodt.txt", 0.03, binary=False, use_omp=True)
status = ubodt_gen.generate_ubodt("../data/chengdu/ubodt.txt", 0.03, binary=False, use_omp=True)
# Binary is faster for both IO and precomputation
# status = ubodt_gen.generate_ubodt("stockholm/ubodt.bin", 0.03, binary=True, use_omp=True)
print(status)

Status: success
Time takes 40.273 seconds



### Create FMM model

In [12]:
### Read UBODT
ubodt = UBODT.read_ubodt_csv("../data/didi/ubodt.txt")
### Create FMM model
fmm_model = FastMapMatch(network,graph,ubodt)

In [13]:
k = 8
radius = 0.003
gps_error = 0.0005
fmm_config = FastMapMatchConfig(k,radius,gps_error)

### Run map matching for gps points

In [14]:
from fmm import GPSConfig, ResultConfig
input_config = GPSConfig()
input_config.file = "../data/didi/val-gps.csv"
input_config.id = "id"
input_config.x = "lng"
input_config.y = "lat"
input_config.gps_point = True

In [15]:
print(input_config.to_string())

gps file : ../data/didi/val-gps.csv
id column : id
geom column : geom
timestamp column : timestamp
x column : lng
y column : lat
GPS point : true



In [16]:
result_config = ResultConfig()
result_config.file = "../data/didi/mr.txt"
result_config.output_config.write_opath = False
print(result_config.to_string())

Result file : ../data/didi/mr.txt
Output fields: cpath mgeom 


In [17]:
status = fmm_model.match_gps_file(input_config, result_config, fmm_config)
print(status)

Status: success
Time takes 261.169 seconds
Total points 67504995 matched 66995293
Total trajectories 362801 matched 359834
Map match percentage 0.992449
Map match speed 256521 points/s 



### Run map matching for wkt

In [20]:
wkt = "LINESTRING(18.05522129697668 59.335820514008,18.050801016520136 59.33614881818502,18.04796860380041 59.33905964310512,18.046037413309687 59.341029232754444,18.049084402750605 59.34262670491522,18.052775122355097 59.34328317853977,18.056809164713496 59.34273611806721,18.059298254679316 59.34006633652867)"
result = fmm_model.match_wkt(wkt,fmm_config)
### Print map matching result
print("Opath ",list(result.opath))
print("Cpath ",list(result.cpath))
print("WKT ",result.mgeom.export_wkt())

Opath  [37, 957, 984, 203, 1263, 545, 1059, 1188]
Cpath  [37, 289, 477, 254, 957, 558, 407, 395, 185, 134, 988, 984, 675, 1230, 203, 814, 1261, 1263, 577, 584, 545, 1059, 818, 1188]
WKT  LINESTRING(18.055113 59.335538,18.055046 59.335564,18.054878 59.335596,18.054529 59.335681,18.054375 59.335721,18.054186 59.335759,18.053463 59.336012,18.053228 59.336087,18.053046 59.336122,18.052861 59.336111,18.051516 59.33605,18.051402 59.33605,18.051291 59.336057,18.051171 59.336099,18.051076 59.336187,18.050981 59.336279,18.050599 59.336665,18.050422 59.336893,18.050374 59.336942,18.050319 59.337004,18.050297 59.33703,18.050276 59.337055,18.050179 59.337166,18.049859 59.33742,18.049394 59.337886,18.049307 59.337973,18.049229 59.338051,18.048391 59.338891,18.048373 59.338909,18.048267 59.339016,18.048188 59.339096,18.048167 59.339117,18.047976 59.33931,18.047688 59.339607,18.047584 59.339706,18.047113 59.340184,18.047023 59.340275,18.046998 59.3403,18.046971 59.340327,18.045839 59.341474,18.04576 

### Create stmatch model

In [11]:
from fmm import Network,NetworkGraph,STMATCH,STMATCHConfig

In [12]:
stmatch_model = STMATCH(network,graph)

In [14]:
k = 4
gps_error = 0.0005
radius = 0.003
vmax = 0.0003
factor = 1.5
stmatch_config = STMATCHConfig(k, radius, gps_error, vmax, factor)

In [15]:
wkt = "LINESTRING(18.05522129697668 59.335820514008,18.050801016520136 59.33614881818502,18.04796860380041 59.33905964310512,18.046037413309687 59.341029232754444,18.049084402750605 59.34262670491522,18.052775122355097 59.34328317853977,18.056809164713496 59.34273611806721,18.059298254679316 59.34006633652867)"
result = model.match_wkt(wkt,fmm_config)
### Print map matching result
print("Opath ",list(result.opath))
print("Cpath ",list(result.cpath))
print("WKT ",result.mgeom.export_wkt())

Opath  [37, 957, 984, 203, 1263, 545, 1059, 1188]
Cpath  [37, 289, 477, 254, 957, 558, 407, 395, 185, 134, 988, 984, 675, 1230, 203, 814, 1261, 1263, 577, 584, 545, 1059, 818, 1188]
WKT  LINESTRING(18.055113 59.335538,18.055046 59.335564,18.054878 59.335596,18.054529 59.335681,18.054375 59.335721,18.054186 59.335759,18.053463 59.336012,18.053228 59.336087,18.053046 59.336122,18.052861 59.336111,18.051516 59.33605,18.051402 59.33605,18.051291 59.336057,18.051171 59.336099,18.051076 59.336187,18.050981 59.336279,18.050599 59.336665,18.050422 59.336893,18.050374 59.336942,18.050319 59.337004,18.050297 59.33703,18.050276 59.337055,18.050179 59.337166,18.049859 59.33742,18.049394 59.337886,18.049307 59.337973,18.049229 59.338051,18.048391 59.338891,18.048373 59.338909,18.048267 59.339016,18.048188 59.339096,18.048167 59.339117,18.047976 59.33931,18.047688 59.339607,18.047584 59.339706,18.047113 59.340184,18.047023 59.340275,18.046998 59.3403,18.046971 59.340327,18.045839 59.341474,18.04576 

### Match GPS data stored in a file¶

In [17]:
from fmm import GPSConfig,ResultConfig

In [18]:
# Define input data configuration
input_config = GPSConfig()
input_config.file = "stockholm/trips.csv"
input_config.id = "id"
print(input_config.to_string())

gps file : stockholm/trips.csv
id column : id
geom column : geom
timestamp column : timestamp
x column : x
y column : y
GPS point : false



In [21]:
# Define output configuration
result_config = ResultConfig()
result_config.file = "stockholm/mr.txt"
result_config.output_config.write_opath = True
print(result_config.to_string())

Result file : stockholm/mr.txt
Output fields: opath cpath mgeom 


In [27]:
status = fmm_model.match_gps_file(input_config, result_config, fmm_config, use_omp = True)
print(status)

Status: success
Time takes 0.001 seconds
Total points 14 matched 14
Total trajectories 2 matched 2
Map match percentage 1
Map match speed 14000 points/s 



In [28]:
status = stmatch_model.match_gps_file(input_config, result_config, stmatch_config, use_omp = True)
print(status)

Status: success
Time takes 0.009 seconds
Total points 14 matched 14
Map match speed 1555.56 points/s 

